In [1]:
!nvidia-smi

Sat Mar 26 10:07:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   33C    P0    33W / 250W |   2059MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"    
import tensorflow as tf

In [3]:
%pwd

'/home/souvic/mounted/btp/vahini/Name2Demographics/Models/Baselines'

In [4]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [5]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [6]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [7]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [8]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [9]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [10]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [12]:
from transformers import pipeline
from sklearn.metrics import classification_report

# print(classification_report(true, labels, digits = 4))

In [13]:
import math
def print_results(rmodel, tokenizer, df, name):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
            
    
    true = []
    actual = list(df['Gender'])
    for elem in actual:
        if elem  == "Male":
            true.append(1)
        else:
            true.append(0)
    
#     true = list(map(int, list(df['Gender'])))
    df['pred'] = labels
    df.to_csv("Models/Baselines/"+name)
    print(classification_report(true, labels, digits = 4))
    return 

In [14]:
import pandas as pd

aieee = pd.read_csv("Models/Baselines/aieee_sample.csv")
er = pd.read_csv("Models/Baselines/er_sample.csv")
cbse = pd.read_csv("Models/Baselines/cbse_sample.csv")

In [29]:
print_results(cbse_model, tokenizer, aieee, "BERTCBSE/aieee_sample.csv")

              precision    recall  f1-score   support

           0     0.8333    0.8744    0.8534       223
           1     0.8947    0.8592    0.8766       277

    accuracy                         0.8660       500
   macro avg     0.8640    0.8668    0.8650       500
weighted avg     0.8674    0.8660    0.8663       500



In [30]:
print_results(cbse_model, tokenizer, er, "BERTCBSE/er_sample.csv")

              precision    recall  f1-score   support

           0     0.7328    0.7522    0.7424       226
           1     0.7918    0.7745    0.7831       275

    accuracy                         0.7645       501
   macro avg     0.7623    0.7634    0.7627       501
weighted avg     0.7652    0.7645    0.7647       501



In [31]:
print_results(cbse_model, tokenizer, cbse, "BERTCBSE/cbse_sample.csv")

              precision    recall  f1-score   support

           0     0.9706    0.9612    0.9659       206
           1     0.9730    0.9796    0.9763       294

    accuracy                         0.9720       500
   macro avg     0.9718    0.9704    0.9711       500
weighted avg     0.9720    0.9720    0.9720       500



In [16]:
print_results(aieee_model, tokenizer, aieee, "BERTAIEEE/aieee_sample.csv")

              precision    recall  f1-score   support

           0     0.9774    0.9686    0.9730       223
           1     0.9749    0.9819    0.9784       277

    accuracy                         0.9760       500
   macro avg     0.9761    0.9753    0.9757       500
weighted avg     0.9760    0.9760    0.9760       500



In [36]:
print_results(aieee_model, tokenizer, er, "BERTAIEEE/er_sample.csv")

              precision    recall  f1-score   support

           0     0.7308    0.7566    0.7435       226
           1     0.7940    0.7709    0.7823       275

    accuracy                         0.7645       501
   macro avg     0.7624    0.7638    0.7629       501
weighted avg     0.7655    0.7645    0.7648       501



In [17]:
print_results(aieee_model, tokenizer, cbse, "BERTAIEEE/cbse_sample.csv")

              precision    recall  f1-score   support

           0     0.9710    0.9757    0.9734       206
           1     0.9829    0.9796    0.9813       294

    accuracy                         0.9780       500
   macro avg     0.9770    0.9777    0.9773       500
weighted avg     0.9780    0.9780    0.9780       500



In [38]:
print_results(er_model, tokenizer, aieee, "BERTER/aieee_sample.csv")

              precision    recall  f1-score   support

           0     0.9149    0.9641    0.9389       223
           1     0.9698    0.9278    0.9483       277

    accuracy                         0.9440       500
   macro avg     0.9424    0.9460    0.9436       500
weighted avg     0.9453    0.9440    0.9441       500



In [39]:
print_results(er_model, tokenizer, er, "BERTER/cbse_sample.csv")

              precision    recall  f1-score   support

           0     0.9391    0.9558    0.9474       226
           1     0.9631    0.9491    0.9560       275

    accuracy                         0.9521       501
   macro avg     0.9511    0.9524    0.9517       501
weighted avg     0.9523    0.9521    0.9521       501



In [46]:
print_results(er_model, tokenizer, cbse, "BERTER/er_sample.csv")

KeyboardInterrupt: 

Muril Models

In [27]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

In [19]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_Muril)

In [25]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_Muril)

In [28]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_Muril)

In [31]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")


In [47]:
print_results(cbse_model, tokenizer, aieee, "MurilCBSE/aieee_sample.csv")

              precision    recall  f1-score   support

           0     0.8761    0.9193    0.8972       223
           1     0.9323    0.8953    0.9134       277

    accuracy                         0.9060       500
   macro avg     0.9042    0.9073    0.9053       500
weighted avg     0.9072    0.9060    0.9062       500



In [48]:
print_results(cbse_model, tokenizer, er, "MurilCBSE/er_sample.csv")

              precision    recall  f1-score   support

           0     0.7309    0.7212    0.7261       226
           1     0.7734    0.7818    0.7776       275

    accuracy                         0.7545       501
   macro avg     0.7522    0.7515    0.7518       501
weighted avg     0.7542    0.7545    0.7543       501



In [49]:
print_results(cbse_model, tokenizer, cbse, "MurilCBSE/cbse_sample.csv")

              precision    recall  f1-score   support

           0     0.9515    0.9515    0.9515       206
           1     0.9660    0.9660    0.9660       294

    accuracy                         0.9600       500
   macro avg     0.9587    0.9587    0.9587       500
weighted avg     0.9600    0.9600    0.9600       500



In [50]:
print_results(aieee_model, tokenizer, aieee, "MurilAIEEE/aieee_sample.csv")

              precision    recall  f1-score   support

           0     0.9636    0.9507    0.9571       223
           1     0.9607    0.9711    0.9659       277

    accuracy                         0.9620       500
   macro avg     0.9622    0.9609    0.9615       500
weighted avg     0.9620    0.9620    0.9620       500



In [51]:
print_results(aieee_model, tokenizer, er, "MurilAIEEE/er_sample.csv")

              precision    recall  f1-score   support

           0     0.7241    0.7434    0.7336       226
           1     0.7844    0.7673    0.7757       275

    accuracy                         0.7565       501
   macro avg     0.7543    0.7553    0.7547       501
weighted avg     0.7572    0.7565    0.7567       501



In [52]:
print_results(aieee_model, tokenizer, cbse, "MurilAIEEE/cbse_sample.csv")

              precision    recall  f1-score   support

           0     0.9617    0.9757    0.9687       206
           1     0.9828    0.9728    0.9778       294

    accuracy                         0.9740       500
   macro avg     0.9723    0.9743    0.9732       500
weighted avg     0.9741    0.9740    0.9740       500



In [53]:
print_results(er_model, tokenizer, aieee, "MurilER/aieee_sample.csv")

              precision    recall  f1-score   support

           0     0.9464    0.9507    0.9485       223
           1     0.9601    0.9567    0.9584       277

    accuracy                         0.9540       500
   macro avg     0.9533    0.9537    0.9535       500
weighted avg     0.9540    0.9540    0.9540       500



In [32]:
print_results(er_model, tokenizer, er, "MurilER/er_sample.csv")

              precision    recall  f1-score   support

           0     0.9356    0.9646    0.9499       226
           1     0.9701    0.9455    0.9576       275

    accuracy                         0.9541       501
   macro avg     0.9529    0.9550    0.9538       501
weighted avg     0.9546    0.9541    0.9541       501



In [33]:
print_results(er_model, tokenizer, cbse, "MurilER/cbse_sample.csv")

              precision    recall  f1-score   support

           0     0.9848    0.9466    0.9653       206
           1     0.9636    0.9898    0.9765       294

    accuracy                         0.9720       500
   macro avg     0.9742    0.9682    0.9709       500
weighted avg     0.9723    0.9720    0.9719       500

